In [1]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt

from mobslim.agents import ActivityType, Plan, InstructionType
from mobslim.expected import SimpleExpectedDurations
from mobslim.listener import EventListener
from mobslim.network import Grid
from mobslim.optimizer import Optimizer
from mobslim.planners.greedy_trip_planner import GreedyTripPlanner
from mobslim.planners.rerouters.simple_rerouter import StaticRouter
from mobslim.sim import Sim

In [2]:
AGENTS = 10
SIZE = 4  # Size of the grid network
MAX_RUNS = 10  # Maximum number of runs for the simulation

# network setup
network = Grid(size=SIZE)
o = network.get_start()
d = network.get_end()
print(network)

X---X---X---X---X
|   |   |   |   |
X---X---X---X---X
|   |   |   |   |
X---X---X---X---X
|   |   |   |   |
X---X---X---X---X
|   |   |   |   |
X---X---X---X---X



In [3]:
# agent setup
plans = {}
for i in range(AGENTS):
    plan = Plan()
    plan.add_activity(type=ActivityType.HOME, location=o, duration=10)
    plan.add_trip(origin=o, destination=d, start_time=0)
    plan.add_activity(type=ActivityType.WORK, location=d, duration=10)
    plan.finish()
    plans[i] = plan

In [4]:
# planner setup
expected_link_durations = SimpleExpectedDurations(network.G)
router = StaticRouter(network=network, expectations=expected_link_durations)
planner = GreedyTripPlanner(plans=plans, router=router, network=network)

# initiate all plans with naive trip estimates
planner.replan(p=1.0)
print(plans[0])

Plan([SOS(), Act(ActivityType.HOME, loc=(0, 0), dur=10), Trip((0, 0)>(4, 4), duration=40.0, route=[(((0, 0), (0, 1)), 5.0, 5.0), (((0, 1), (0, 2)), 5.0, 5.0), (((0, 2), (0, 3)), 5.0, 5.0), (((0, 3), (0, 4)), 5.0, 5.0), (((0, 4), (1, 4)), 5.0, 5.0), (((1, 4), (2, 4)), 5.0, 5.0), (((2, 4), (3, 4)), 5.0, 5.0), (((3, 4), (4, 4)), 5.0, 5.0)]), Act(ActivityType.WORK, loc=(4, 4), dur=10), EOS()])


In [5]:
# event planner setup
listener = EventListener()

# simulation setup
sim = Sim(
    network=network,
    listener=listener,
)

# optimizer setup
optimizer = Optimizer(sim=sim, plans=plans, planner=planner)
events = optimizer.run(max_runs=MAX_RUNS, p=0.2)

--- Starting optimization ---


AttributeError: 'NoneType' object has no attribute 'keys'

In [ ]:
def build_traces(events):
    agent_traces = {}
    start_link_traverse = {}

    for time, agent_id, instruction in events:
        time = int(time)
        if instruction[0] == InstructionType.SOS:
            agent_traces[agent_id] = []
        elif instruction[0] == InstructionType.EnterActivity:
            agent_traces[agent_id].append((time, instruction[2]))
        elif instruction[0] == InstructionType.EnterLink:
            start_link_traverse[agent_id] = time
        elif instruction[0] == InstructionType.ExitLink:
            (x0, y0), (x1, y1) = instruction[2]
            start_time = start_link_traverse[agent_id]
            duration = time - start_time
            dx, dy = x1 - x0, y1 - y0
            for t in range(duration):
                p = t/duration
                x = x0 + p * dx
                y = y0 + p * dy
                agent_traces[agent_id].append((start_time + t, (x,y)))
        
    return agent_traces, time

In [ ]:
agent_traces, finish_time = build_traces(events)

plt.rcParams["animation.html"] = "jshtml"
plt.ioff()

fig, ax = plt.subplots(figsize=(6, 6))
ax.set_axis_off()

lines = [ax.plot([u[0],v[0]], [u[1], v[1]], color="grey") for u, v in network.G.edges()]
starts = {idx: trace.pop(0)[1] for idx, trace in agent_traces.items()}
scatters = {idx: ax.scatter(loc[0], loc[1], color="red") for idx, loc in starts.items()}

def animate(frame):
    current_time = frame
    for idx, locations in agent_traces.items():
        if not locations:
            continue
        time, pos = locations[0]
        if time <= current_time:
            _, pos = locations.pop(0)
            scatters[idx].set_offsets(pos)

    return ax


frame_rate = 1  # frames per second
ani = animation.FuncAnimation(
    fig, animate, frames=finish_time
)
ani